In [1]:
import os,sys
import matplotlib.pyplot as plt
#import pandas as pd  # AC got a 'no module named pandas' error.  Do we need it?
import numpy as np
import math
import csv
import scipy

# Eventually we'll have to find a common pathway, for now just change if this is commented out.
#sys.path.insert(0,'/Users/dtn1/gsas2full/GSASII/') # needed to "find" GSAS-II modules
sys.path.insert(0,'/Users/creuzige/gsas2full/envs/gsas2pkg/GSASII/') # needed to "find" GSAS-II modules

import GSASIIscriptable as G2sc

GSAS-II binary directory: /Users/creuzige/gsas2full/envs/gsas2pkg/GSASII/bindist
6 values read from config file /Users/creuzige/gsas2full/envs/gsas2pkg/GSASII/config.py


#### Convert to keV to use FPcalc

In [2]:
def LambdaAng2keV(Lambda):
    '''
    Convert between wavelength in angstroms and energy in keV
    '''
    keV=scipy.constants.h*scipy.constants.c/(Lambda*1e-10*scipy.constants.value("electron volt")*1000)
    return keV

### Hardcoded values
- for now, eventually these will be from the data

In [3]:
# Theta from Ferrite 110, from Jatczak
Theta= 44.60/2 # degrees
# Theta= 44.60/2 # degrees
print(Theta)


# Cu radiation from Jatczak
Lambda=1.54178 # angstrom

# Cu radiation from GSASII
#Lambda=1.5405 # angstrom, 8.0477 keV

# K-beta peak
#Lambda=1.392 # angstrom, 8.0477 keV
print(Lambda)

# Calculate the sin(theta)/lambda value
sTL=math.sin(math.radians(Theta))/Lambda
print(sTL)




22.3
1.54178
0.24611563227503605


### Phase Composition values
- Trying to use IUCR .cif definitions
- Overall composition entered by the user
- Should check if atom_site_occupancy equals 1?

In [4]:
Phase={
    'Fe': {
    "atom_site_type_symbol": "Fe",# CIF term
    "atom_site_occupancy":0.8, # CIF term
    "atom_site_occupancy_uncert":0.08 # estimage
    },
    'Ni': {
    "atom_site_type_symbol": "Ni",# CIF term
    "atom_site_occupancy":0.1, # CIF term
    "atom_site_occupancy_uncert":0.01 #      
    },
    'Cr': {
    "atom_site_type_symbol": "Cr",# CIF term
    "atom_site_occupancy": 0.1, # CIF term
    "atom_site_occupancy_uncert":0.01 #  
    }
}

In [13]:
TotalScattering=0

for key in Phase:
    # Find the coefficients to calculate scattering factor
    FormFactor=G2sc.G2elem.GetFormFactorCoeff(Phase[key]["atom_site_type_symbol"])[0]
    #print(FormFactor)
    # Use coefficients to calculate the first term scattering
    ScatteringFactor0=(G2sc.G2elem.ScatFac(FormFactor, (sTL**2))[0])
    print(ScatteringFactor0)
    
    # Find the electron orbital structures
    
    Orbitals=G2sc.G2elem.GetXsectionCoeff(Phase[key]["atom_site_type_symbol"])
    (f_prime, f_2prime, mu)= G2sc.G2elem.FPcalc(Orbitals,LambdaAng2keV(Lambda))
    print(f_prime)
    print(Phase[key]["atom_site_occupancy"])
    TotalScattering=TotalScattering+(ScatteringFactor0+f_prime)*Phase[key]["atom_site_occupancy"]
    
    # Need to add mechanism to calculate the scattering uncertainty
    
    print(TotalScattering)
    print()

18.490979644372967
-1.1413067330877997
0.8
13.879738329028136

20.531584781362227
-2.98321132822868
0.1
15.634575674341491

16.694110542807746
-0.16619379065626416
0.1
17.28736734955664



## Prior attempts, not used
- Reading in the table directly

In [ ]:
import importlib.util
spec = importlib.util.spec_from_file_location("atmdata.py",
                                              "/Users/creuzige/gsas2full/envs/gsas2pkg/GSASII/atmdata.py")
AtomDataModule = importlib.util.module_from_spec(spec)
spec.loader.exec_module(AtomDataModule)


In [ ]:
print(AtomDataModule.XrayFF['Fe'])
print(AtomDataModule.XrayFF['Fe']['fa'])

In [ ]:
print(AtomDataModule.XrayFF['Fe']['fa'][1])

In [ ]:
# Cu radiation Ferrite 110 values from Jatczak
#Theta= 44.60/2 # degrees
Theta= 44.10/2 # degrees
#Lambda=1.54178 # angstrom

#Values in GSASII
#Lambda=1.5405 # angstrom, 8.0477 keV
Lambda=1.392 # angstrom, 8.0477 keV



sTL=math.sin(math.radians(Theta))/Lambda
print(sTL)

AtomicScatteringCoeff(AtomDataModule.XrayFF['Fe'],sTL)


In [ ]:
def AtomicScatteringCoeff(AtomDict, SinTheta_Lambda):
    FF=0
    for i in [0,1,2,3]:
        FF=FF+AtomDict['fa'][i]*math.exp(-AtomDict['fb'][i]*SinTheta_Lambda**2)
    FF=FF+AtomDict['fc']
    return FF